## Pool Simulator!

We're making a retro pool simulator in which a user can input an angle and force to the cue ball and play a turn based game of pool.

In [ ]:
pip install import_ipynb

In [1]:
# imports

import numpy as np
import matplotlib.pyplot as plt
from __future__ import annotations
from collections.abc import Callable
import import_ipynb
from BallClass import Ball
from StateClass import State

# Everything Below is Eli's work

In [ ]:
class Player():
    '''
    I felt the need for a Player class, to keep track of each players balls left to win,
    whether they're down to the right ball, have won or lost the game. This will be use in the main loop
    so we can just say what player did in each turn.
    '''
    name: str
    ID: int
    team: str
    balls: list[int]
    ballsLeft: int
    eight: bool
    winner: bool # will break the loop
    loser: bool # will break the loop
    
    def __init__(self, name: int, ID: int):
        self.name = name
        self.ID = ID
        self.team = "undefined"
        self.balls = []
        self. ballsLeft = len(self.balls)
        self.downToTheEight = False
        self.winner = False
        self.loser = False
    
    # find the initial list of balls this player has to pocket to win
    def balls_init(team: str):
        assert team == "stripes" or team == "solids", "Not a valid team."
        
        '''
        doesn't currently account for balls pocketed before team assignment 
        might have to take in the list of all balls from state
        '''
        
        if team == "solids":
            self.balls = [i for i in range(1,8)] #all solids
        else:
            self.balls = [i for i in range(9,16)] #all stripes

    # initialize the player's list of balls to pocket and team class variable    
    def set_team(team: str):
        self.team = team
        self.balls = self.balls_init(team)
        self.ballsLeft = len(self.balls)
        
    #what team is this ball?    
    def ball_whatTeam(ID: int):
        assert ID >= 0 and ID <= 15, "Not a valid ball ID."
        
        if ID >= 1 and ID <= 7:
            return "solid"
        elif ID >= 9 and ID <= 15:
            return "stripe"
        elif ID == 8:
            return "eight"
        elif ID == 0:
            return "cue"
        
    def scratch():
        '''
        idk how to implement a scratch yet
        next turn  logic goes here too, if you scratch it's not your turn again
        '''
        ...
        
    #if a player runs out of balls they either win or are now down to the eight
    def ranOutOfBalls(self):
        if self.downToTheEight:
            self.winner = True
        else:
            self.DownToTheEight = True
            self.balls.append(8)
            ''' player plays again, next turn logic'''
            
    # what happens when you pocket each type of ball
    def pocket(self, ball_ID: int):
        
        ball_team: str = self.ball_whatTeam(ball_ID)
        
        if ball_team == self.team:
            self.balls.remove(ball_ID)
            self.ballsLeft = len(self.balls)
            ''' if you score you play again, next turn logic'''
        
        elif ball_team == "eight":
            if self.DownToTheEight:
                self.winner == True
            else:
                self.loser == True #pocket the eight too early
        
        # wrong team or cue means it scratched
        else:
            self.scratch()

# Everything Below is Jose's work on Plotting

In [ ]:
# Get Ball Color for Plot
def getBallColor(ball: Ball):

    colors = {0: "white", 1: "yellow", 2: "blue", 3: "red", 4: "purple", 5: "darkorange",6: "limegreen", 7: "brown", 8: "black", 9: "yellow", 10: "blue", 11: "red", 12: "purple", 13: "darkorange", 14: "limegreen", 15: "brown"}

    return colors.get(ball.ID)



In [ ]:
ball_test = Ball(1,0.06,1,1,0,0)

In [ ]:
# Example Ball and State Variables to implement Matplotlib
balls = []
for i in range(0, 16):
    balls += [Ball(i, 0.06, np.random.uniform(-1.21, 1.21), np.random.uniform(-2.48, 2.48), 0, 0)]

print(balls)


In [ ]:
# Create Plot
W_TABLE = 1.27
H_TABLE = 2.54
fig, ax = plt.subplots(figsize=(W_TABLE*6,H_TABLE*6))
plt.rcParams["hatch.linewidth"] = 4

ax.margins(0.3)
ax.set_facecolor("green")
ax.set_ylim(-H_TABLE, H_TABLE)
ax.set_xlim(-W_TABLE, W_TABLE)

for ball in balls:
     color = getBallColor(ball)

     if ball.team == "stripe":
         plotBall = plt.Circle(ball.p, ball.radius, facecolor= color, edgecolor ="white", hatch =r"-")
     else:
         plotBall = plt.Circle(ball.p, ball.radius, facecolor= color, edgecolor ="white")

     ax.add_patch(plotBall)

ax.get_yaxis().set_visible(False)
ax.get_xaxis().set_visible(False)

plt.show()

## Future Implementations:
#  Animation
#  Text Showing Points/Whose Turn is Next
#  More Detailed Table Outline/Pockets


In [ ]:
'''stretch goals: 

 - non-regulation non-euclidean pool tables, using a table class that takes in the number of sides and their lengths'''